## Imports

In [3]:
import math
import numpy as np
import pandas
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

# Preparando el dataset de entrenamiento

In [4]:
data_train = pandas.read_csv('data/perrosTrainAllComplete.csv')

data_train


,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,1,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,2,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,3,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,4,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,5,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,1910,Adulto,Chico,Macho,Bicolor,Dorado,Blanco,NaN,Corto,Marron oscuro,Mediano,Mediano,Mediano,Caidas
316,1919,Cachorro,Mediano,Macho,Tricolor,Negro,Blanco,Marron,Largo,Marron oscuro,Corto,Corto,Mediano,Caidas
317,1940,Cachorro,Mediano,Macho,Liso,Marron,NaN,NaN,Corto,Marron claro,Largo,Mediano,Cortas,Caidas
318,1950,Adulto,Chico,Hembra,Bicolor,Negro,Marron,NaN,Largo,Marron oscuro,Mediano,Corto,Largas,Caidas


### Obtenemos la cantidad de valores vacios

In [5]:
null_value_stats = data_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Color de pelaje 2    129
Color de pelaje 3    291
dtype: int64

### Lleno los nulos con strings vacios y quito la columna "Mascota"

In [6]:
data_train.fillna("NaN", inplace=True)
prepared_data_train = data_train.drop('Mascota', axis=1)
prepared_data_train

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Mediano,Macho,Bicolor,Blanco,Negro,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
1,Cachorro,Mediano,Hembra,Bicolor,Blanco,Marron,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Caidas
2,Adulto,Mediano,Macho,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Largas,Paradas
3,Adulto,Mediano,Hembra,Liso,Dorado,NaN,NaN,Corto,Marron oscuro,Largo,Largo,Cortas,Paradas
4,Cachorro,Chico,Hembra,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,Corto,Cortas,Caidas
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Adulto,Chico,Macho,Bicolor,Dorado,Blanco,NaN,Corto,Marron oscuro,Mediano,Mediano,Mediano,Caidas
316,Cachorro,Mediano,Macho,Tricolor,Negro,Blanco,Marron,Largo,Marron oscuro,Corto,Corto,Mediano,Caidas
317,Cachorro,Mediano,Macho,Liso,Marron,NaN,NaN,Corto,Marron claro,Largo,Mediano,Cortas,Caidas
318,Adulto,Chico,Hembra,Bicolor,Negro,Marron,NaN,Largo,Marron oscuro,Mediano,Corto,Largas,Caidas


### Imprimo los tipos de cada columna

In [7]:
print(prepared_data_train.dtypes)


Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


### Obtengo la lista de etiquetas de las categorias

In [8]:
dataset_labels = prepared_data_train.columns.to_list()
dataset_labels

['Edad',
 'Tamaño',
 'Sexo',
 'Patron de pelaje',
 'Color de pelaje 1',
 'Color de pelaje 2',
 'Color de pelaje 3',
 'Largo de pelaje',
 'Color de ojos',
 'Largo de hocico',
 'Largo de cola',
 'Largo de orejas',
 'Tipo de orejas']

In [263]:
mascotas_train_ids = data_train.Mascota
print(mascotas_train_ids)

0         1
1         2
2         3
3         4
4         5
       ... 
315    1910
316    1919
317    1940
318    1950
319    1954
Name: Mascota, Length: 320, dtype: int64


Obtengo las categorias en base a las que no son numericas, en este caso todas son categorias

In [10]:
categorical_features_indices = np.where(prepared_data_train.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Preparando el dataset de testeo

In [243]:
dataset_test = pandas.read_csv('./data/perrosTestComplete.csv')
dataset_test.head()

null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

print(prepared_dataset_test.dtypes)

mascotas_test_ids = dataset_test.Mascota
mascotas_test_ids.head()

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


0    1048
Name: Mascota, dtype: int64

In [244]:
dataset_test = pandas.read_csv('./data/perrosTestComplete.csv')
dataset_test.head()

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,1048,Adulto,Grande,Macho,Bicolor,Negro,Dorado,NaN,Mediano,Marron claro,Largo,Mediana,Mediano,Caidas


In [245]:
null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Color de pelaje 3    1
dtype: int64

In [246]:
dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Grande,Macho,Bicolor,Negro,Dorado,NaN,Mediano,Marron claro,Largo,Mediana,Mediano,Caidas


In [247]:
print(prepared_dataset_test.dtypes)

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


In [248]:
mascotas_test_ids = dataset_test.Mascota
mascotas_test_ids.head()

0    1048
Name: Mascota, dtype: int64

# Creando y entrenando el modelo

### Creacion del modelo

In [181]:
train_dataset = Pool(data=prepared_data_train,
                     label=mascotas_train_ids,
                        cat_features=categorical_features_indices)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=3,
                           loss_function='MultiClass')
# Fit model


### Cargando un model

In [ ]:
# modelImport = CatBoostClassifier().load_model("models/model1.cbm", format='cbm')

# **Entrenamiento**

In [182]:
model.fit(train_dataset)

0:	learn: 5.7496664	total: 14.7s	remaining: 24m 18s
1:	learn: 5.7190176	total: 28.7s	remaining: 23m 26s
2:	learn: 5.5673909	total: 40.5s	remaining: 21m 49s
3:	learn: 5.5088715	total: 52.3s	remaining: 20m 54s
4:	learn: 5.4946975	total: 58.6s	remaining: 18m 33s
5:	learn: 5.4702178	total: 1m 10s	remaining: 18m 32s
6:	learn: 5.4364767	total: 1m 23s	remaining: 18m 24s
7:	learn: 5.3238921	total: 1m 34s	remaining: 18m 12s
8:	learn: 5.1142671	total: 1m 49s	remaining: 18m 22s
9:	learn: 5.0617897	total: 2m	remaining: 18m 1s
10:	learn: 5.0343091	total: 2m 10s	remaining: 17m 39s
11:	learn: 5.0221683	total: 2m 23s	remaining: 17m 33s
12:	learn: 5.0172048	total: 2m 32s	remaining: 16m 57s
13:	learn: 5.0067601	total: 2m 40s	remaining: 16m 25s
14:	learn: 4.8103489	total: 2m 48s	remaining: 15m 53s
15:	learn: 4.7913372	total: 2m 56s	remaining: 15m 29s
16:	learn: 4.7647270	total: 3m 7s	remaining: 15m 16s
17:	learn: 4.7477545	total: 3m 16s	remaining: 14m 55s
18:	learn: 4.5958262	total: 3m 25s	remaining: 14m

# Guardando el model

In [183]:
model.save_model("models/modelDepth3Ite100.cbm",
                 format="cbm",
                 export_parameters=None,
                 pool=None)

model.save_model("models/modelDepth3Ite100.json",
                 format="json",
                 export_parameters=None,
                 pool=None)


# Generar predicciones

In [310]:
eval_dataset = Pool(data=prepared_dataset_test,
                    label=mascotas_test_ids,
                    cat_features=categorical_features_indices)


In [311]:
# Get predicted classes
preds_class = model.predict(eval_dataset)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_dataset)

In [312]:
print(preds_class)

[[4282]]


## Probabilidades
Un array por cada mascota de testeo

In [309]:
print(preds_proba[0])


[0.01988141 0.00260165 0.00250418 0.00023512 0.00186216 0.00485999
 0.00077865 0.0024341  0.0001856  0.00133064 0.00210516 0.00226266
 0.01341249 0.00079149 0.00033876 0.00083829 0.00316575 0.01221865
 0.00224798 0.00053345 0.00053444 0.00050941 0.00379952 0.00538706
 0.0026165  0.03099027 0.00969176 0.00319387 0.00275807 0.0200625
 0.00481857 0.00295782 0.00100909 0.00218109 0.01759742 0.00118598
 0.00315919 0.00084496 0.00079361 0.00854237 0.00388904 0.00318187
 0.00603114 0.00303624 0.00244161 0.00020978 0.00277752 0.00318115
 0.00934317 0.0051108  0.00036488 0.00082945 0.00257983 0.00216876
 0.00497693 0.00269205 0.00041955 0.00830936 0.00046734 0.00025264
 0.00144763 0.0001443  0.00112323 0.02302094 0.00109082 0.00319841
 0.0023076  0.00152262 0.0045411  0.00310908 0.02904142 0.01765102
 0.00048671 0.00949561 0.00200197 0.00192667 0.00393935 0.00192679
 0.00052841 0.00871209 0.00033004 0.00116728 0.00076943 0.00085291
 0.00029293 0.00053086 0.00717132 0.00156877 0.00107952 0.00047

### Supuestas predicciones

In [319]:
mascota_numero = 0

for prediccion in preds_class:
    probabilidad = math.trunc((preds_proba[mascota_numero][prediccion.index()])*100)
    print("-- Mascota "+str(mascotas_test_ids[mascota_numero])+" es similar a la mascota "+str(prediccion.item())
        +" => "+str(probabilidad)+"% de probabilidad")
    # print(str(preds_proba[mascota_numero]))
    print(dataset_test.loc[mascota_numero:mascota_numero, "Edad":"Tipo de orejas"].values)
    print(prepared_data_train.loc[prediccion.item():prediccion.item(), "Edad":"Tipo de orejas"].values)
    mascota_numero+=1


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [241]:
class Mascota:
    def __init__(self,index, id, prob):
        self.id = id
        self.index = index
        self.prob = prob

    def to_string(self):
        return "{} : {} => {}".format(self.index,self.id,self.prob)

    def get_id(self):
        return self.id
    
    def get_prob(self):
        return self.prob

    def get_index(self):
        return self.index

In [296]:
def keyValue():
    mascota_test_index = 0
    # mascota_probs = preds_proba[0]

    mascota_test_id = str(mascotas_test_ids[mascota_test_index].item())
    print(mascota_test_id)
    
    mascotas_probs = []

    for prob_index in range(0, len(preds_proba[mascota_test_index])):
        probabilidad = preds_proba[mascota_test_index][prob_index]*100
        # print(preds_proba[mascota_test_index][41])
        # print(mascotas_train_ids[prob_index],"=>","{0:.2f}".format(probabilidad),"%")
        mascotas_probs.append(
            Mascota(prob_index,mascotas_train_ids[prob_index], probabilidad))
        
    print(len(mascotas_probs))

    def orderProb(n):
        return n.get_prob()

    def orderId(n):
        return n.get_id()

    def orderIndex(n):
        return n.get_index()

    mascotas_probs.sort(key=orderProb, reverse=True)

    for mascota in mascotas_probs:
        print(mascota.to_string())

keyValue()


1048
320
235 : 4416 => 3.8889843827318487
25 : 26 => 3.099027229571469
70 : 71 => 2.904142398307329
63 : 67 => 2.3020944442145197
29 : 30 => 2.0062495846213433
0 : 1 => 1.9881405482132422
176 : 6012 => 1.9777778532330148
137 : 3950 => 1.8782777259934194
71 : 72 => 1.7651021738883474
34 : 35 => 1.7597422015686468
116 : 3789 => 1.5320072626690397
306 : 1851 => 1.400983889591296
287 : 1702 => 1.3850746272542631
314 : 1908 => 1.372942085603454
167 : 6004 => 1.3685033642075208
12 : 13 => 1.341249099320969
205 : 1477 => 1.2513029224693792
98 : 3528 => 1.2228250288211637
17 : 18 => 1.2218646724036
201 : 4352 => 1.1810594147545979
106 : 3601 => 0.986562794144793
26 : 27 => 0.969176369510197
319 : 1954 => 0.9496065932239675
73 : 73 => 0.9495606952896647
274 : 4535 => 0.94711112152172
48 : 49 => 0.934316740866685
272 : 4531 => 0.9206943953942287
236 : 1529 => 0.8850834218589194
174 : 1431 => 0.8785714044669207
79 : 1067 => 0.8712085839990841
39 : 40 => 0.8542368778496771
57 : 1007 => 0.830936346